# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f0770ae0f10>
├── 'a' --> tensor([[-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356]])
└── 'x' --> <FastTreeValue 0x7f0770ae0f70>
    └── 'c' --> tensor([[-0.2790, -0.0511, -0.8639, -0.4985],
                        [-0.1105, -0.7906, -0.2839,  0.9364],
                        [-1.7278, -0.4639, -0.1368, -0.6824]])

In [4]:
t.a

tensor([[-0.6042,  1.1408,  0.8072],
        [-0.3247, -1.0783,  0.6356]])

In [5]:
%timeit t.a

47.4 ns ± 0.325 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f0770ae0f10>
├── 'a' --> tensor([[ 0.3849, -0.6936,  0.1456],
│                   [ 0.7167,  0.4908, -0.2481]])
└── 'x' --> <FastTreeValue 0x7f0770ae0f70>
    └── 'c' --> tensor([[-0.2790, -0.0511, -0.8639, -0.4985],
                        [-0.1105, -0.7906, -0.2839,  0.9364],
                        [-1.7278, -0.4639, -0.1368, -0.6824]])

In [7]:
%timeit t.a = new_value

52.8 ns ± 0.0805 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.6042,  1.1408,  0.8072],
               [-0.3247, -1.0783,  0.6356]]),
    x: Batch(
           c: tensor([[-0.2790, -0.0511, -0.8639, -0.4985],
                      [-0.1105, -0.7906, -0.2839,  0.9364],
                      [-1.7278, -0.4639, -0.1368, -0.6824]]),
       ),
)

In [10]:
b.a

tensor([[-0.6042,  1.1408,  0.8072],
        [-0.3247, -1.0783,  0.6356]])

In [11]:
%timeit b.a

42 ns ± 0.744 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.2927,  0.6275, -1.2657],
               [-0.6560, -0.2843,  1.0725]]),
    x: Batch(
           c: tensor([[-0.2790, -0.0511, -0.8639, -0.4985],
                      [-0.1105, -0.7906, -0.2839,  0.9364],
                      [-1.7278, -0.4639, -0.1368, -0.6824]]),
       ),
)

In [13]:
%timeit b.a = new_value

367 ns ± 4.36 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

638 ns ± 6.91 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

8.4 µs ± 148 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

127 µs ± 621 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

125 µs ± 659 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f068bf8de20>
├── 'a' --> tensor([[[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]],
│           
│                   [[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]],
│           
│                   [[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]],
│           
│                   [[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]],
│           
│                   [[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]],
│           
│                   [[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]],
│           
│                   [[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]],
│           
│                   [[-0.6042,  1.1408,  0.8072],
│                    [-0.3247, -1.0783,  0.6356]]])
└── 'x' --> <FastTreeValue 0x7f07850e0c40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

23.6 µs ± 271 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f0770b492b0>
├── 'a' --> tensor([[-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356],
│                   [-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356],
│                   [-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356],
│                   [-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356],
│                   [-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356],
│                   [-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356],
│                   [-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356],
│                   [-0.6042,  1.1408,  0.8072],
│                   [-0.3247, -1.0783,  0.6356]])
└── 'x' --> <FastTreeValue 0x7f068bf770d0>
    └── 'c' --> tensor([[-0.2790, -0.0511, -0.8639, -0.4985],
                        [-0.1105, -0.7906, -0.2839,  0.9364],
                 

In [23]:
%timeit t_cat(trees)

22.1 µs ± 60.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

50 µs ± 205 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2790, -0.0511, -0.8639, -0.4985],
                       [-0.1105, -0.7906, -0.2839,  0.9364],
                       [-1.7278, -0.4639, -0.1368, -0.6824]],
              
                      [[-0.2790, -0.0511, -0.8639, -0.4985],
                       [-0.1105, -0.7906, -0.2839,  0.9364],
                       [-1.7278, -0.4639, -0.1368, -0.6824]],
              
                      [[-0.2790, -0.0511, -0.8639, -0.4985],
                       [-0.1105, -0.7906, -0.2839,  0.9364],
                       [-1.7278, -0.4639, -0.1368, -0.6824]],
              
                      [[-0.2790, -0.0511, -0.8639, -0.4985],
                       [-0.1105, -0.7906, -0.2839,  0.9364],
                       [-1.7278, -0.4639, -0.1368, -0.6824]],
              
                      [[-0.2790, -0.0511, -0.8639, -0.4985],
                       [-0.1105, -0.7906, -0.2839,  0.9364],
                       [-1.7278, -0.4639, -0.1368, -0.6824]],

In [26]:
%timeit Batch.stack(batches)

62.2 µs ± 371 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2790, -0.0511, -0.8639, -0.4985],
                      [-0.1105, -0.7906, -0.2839,  0.9364],
                      [-1.7278, -0.4639, -0.1368, -0.6824],
                      [-0.2790, -0.0511, -0.8639, -0.4985],
                      [-0.1105, -0.7906, -0.2839,  0.9364],
                      [-1.7278, -0.4639, -0.1368, -0.6824],
                      [-0.2790, -0.0511, -0.8639, -0.4985],
                      [-0.1105, -0.7906, -0.2839,  0.9364],
                      [-1.7278, -0.4639, -0.1368, -0.6824],
                      [-0.2790, -0.0511, -0.8639, -0.4985],
                      [-0.1105, -0.7906, -0.2839,  0.9364],
                      [-1.7278, -0.4639, -0.1368, -0.6824],
                      [-0.2790, -0.0511, -0.8639, -0.4985],
                      [-0.1105, -0.7906, -0.2839,  0.9364],
                      [-1.7278, -0.4639, -0.1368, -0.6824],
                      [-0.2790, -0.0511, -0.8639, -0.4985],
                   

In [28]:
%timeit Batch.cat(batches)

117 µs ± 307 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

266 µs ± 3.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
